# ML for Bioinformatics
## Project Phase 2 - Improving DeepDTA

---

Name: Yasamin Tabatabaee

Student No.: 95104866

---

## 1. Analysis of DeepDTA Model

a summary of how it works and the architecture

### 1.1 Positive Points:

1. DeepDTA uses raw sequences (character representations of proteins and drugs) directly as an input to the model, compared to previous state-of-the art drug-target prediction methods that use manual feature engineering. By using raw data, this model achieves similar or higher performance while omiting feature extraction difficulties and overheads. [2]

2. This model uses 1-dimensional representation of sequences and compounds instead of 2 or 3 dimensional structures and features. 

### 1.2 Negative Points:

1. DeepDTA and in general DL-based models are data hungry and require large amounts of high quality data to work well. Reliability and performance of this model is largely dependent on the data used. [1]

2. Features in DL-based models are generated automatically in constrast to other ML-based models that use hand-engineered features. Therefore, DTA results are not easily interpretable and do not take into consideration the biological or chemical aspects of the problem and different statistical analysis of the data. [1]

3. DeepDTA uses CNN blocks to learn feature representations in its architecture. The paper reports that using two CNN blocks instead of one increases the performance significantly. This suggests that CNN have not been able to learn the order relationship in the protein sequence captured in its structural data. 

## 2. Resolving the Negative Points

## 3. Preprocessing

## 4. Implementation

### 4.1. Setup 

#### 4.1.1 Importing libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pickle
import json
import itertools
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm, trange
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from IPython import display
import collections
import math
import tensorflow as tf
from collections import OrderedDict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


#### 4.1.2 Loading Data



In [5]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My\ Drive
!unzip deepdta_data.zip

KeyboardInterrupt: ignored

#### 4.1.3 Reading Data



In [ ]:
# settings

batch_size = 256
davis_smi_len = 85
kiba_smi_len = 100
davis_protein_len = 1200
kiba_protein_len = 1000
kiba_dataset_path = 'data/kiba/'
davis_dataset_path = 'data/davis/'

CHARPROTSET = { "A": 1, "C": 2, "B": 3, "E": 4, "D": 5, "G": 6, 
				"F": 7, "I": 8, "H": 9, "K": 10, "M": 11, "L": 12, 
				"O": 13, "N": 14, "Q": 15, "P": 16, "S": 17, "R": 18, 
				"U": 19, "T": 20, "W": 21, 
				"V": 22, "Y": 23, "X": 24, 
				"Z": 25 }

CHARISOSMISET = {"#": 29, "%": 30, ")": 31, "(": 1, "+": 32, "-": 33, "/": 34, ".": 2, 
				"1": 35, "0": 3, "3": 36, "2": 4, "5": 37, "4": 5, "7": 38, "6": 6, 
				"9": 39, "8": 7, "=": 40, "A": 41, "@": 8, "C": 42, "B": 9, "E": 43, 
				"D": 10, "G": 44, "F": 11, "I": 45, "H": 12, "K": 46, "M": 47, "L": 13, 
				"O": 48, "N": 14, "P": 15, "S": 49, "R": 16, "U": 50, "T": 17, "W": 51, 
				"V": 18, "Y": 52, "[": 53, "Z": 19, "]": 54, "\\": 20, "a": 55, "c": 56, 
				"b": 21, "e": 57, "d": 22, "g": 58, "f": 23, "i": 59, "h": 24, "m": 60, 
				"l": 25, "o": 61, "n": 26, "s": 62, "r": 27, "u": 63, "t": 28, "y": 64}

In [ ]:
# reading test, train, val indices

kiba_train_indices = list(itertools.chain.from_iterable(json.load(open(kiba_dataset_path + "folds/train_fold_setting1.txt"))[0:4]))
kiba_val_indices = json.load(open(kiba_dataset_path + "folds/train_fold_setting1.txt"))[4]
kiba_test_indices = json.load(open(kiba_dataset_path + "folds/test_fold_setting1.txt"))

davis_train_indices = list(itertools.chain.from_iterable(json.load(open(davis_dataset_path + "folds/train_fold_setting1.txt"))[0:4]))
davis_val_indices = json.load(open(davis_dataset_path + "folds/train_fold_setting1.txt"))[4]
davis_test_indices = json.load(open(davis_dataset_path + "folds/test_fold_setting1.txt"))

In [ ]:
# functions for reading data
# partially taken from https://github.com/hkmztrk/DeepDTA/blob/master/source/datahelper.py

def label_smiles(line, MAX_SMI_LEN, smi_ch_ind):
    X = np.zeros(MAX_SMI_LEN)
    for i, ch in enumerate(line[:MAX_SMI_LEN]):
        X[i] = smi_ch_ind[ch]
    return X

def label_sequence(line, MAX_SEQ_LEN, smi_ch_ind):
    X = np.zeros(MAX_SEQ_LEN)
    for i, ch in enumerate(line[:MAX_SEQ_LEN]):
        X[i] = smi_ch_ind[ch]
    return X

def read_dataset(dataset_path, log_space=False):
    proteins = json.load(open(dataset_path + "proteins.txt"), object_pairs_hook=OrderedDict)
    ligands = json.load(open(dataset_path + "ligands_can.txt"), object_pairs_hook=OrderedDict)
    Y = pickle.load(open(dataset_path + "Y", "rb"), encoding='latin1')
    if log_space:
        Y = -(np.log10(Y/(math.pow(10,9))))
    label_row_inds, label_col_inds = np.where(np.isnan(Y)==False) 
    return proteins, ligands, Y, label_row_inds, label_col_inds

def parse_data(proteins, ligands, protein_len, smi_len): 
    XD = []
    XT = []
    for t in proteins.keys():
        XT.append(label_sequence(proteins[t], protein_len, CHARPROTSET))
    for d in ligands.keys():
        XD.append(label_smiles(ligands[d], smi_len, CHARISOSMISET))
    return XD, XT

In [ ]:
class DeepDTADataset(Dataset):
    def __init__(self, dataset_path, indices, protein_len, smi_len, log_space=False):
        super(DeepDTADataset, self).__init__()
        self.indices = indices
        self.proteins, self.ligands, self.Y, self.label_row_inds, self.label_col_inds = read_dataset(dataset_path, log_space)
        self.Y = np.asarray(self.Y).astype(float)
        self.XD, self.XT =  parse_data(self.proteins, self.ligands, protein_len, smi_len) 
        self.XD = np.asarray(self.XD)
        self.XT = np.asarray(self.XT)

    def __getitem__(self, index):
        if torch.is_tensor(index):
           index = index.tolist()
        i = self.indices[index]
        row_index = self.label_row_inds[i]
        col_index = self.label_col_inds[i]   
        return (self.XD[row_index], self.XT[col_index], self.Y[row_index][col_index])

    def __len__(self):
        return len(self.indices)                     

In [ ]:
# Loading KIBA dataset

kiba_train_set = DeepDTADataset(kiba_dataset_path, 
                                kiba_train_indices,
                                kiba_protein_len,
                                kiba_smi_len,
                                log_space=False)
kiba_val_set = DeepDTADataset(kiba_dataset_path, 
                                kiba_val_indices,
                                kiba_protein_len,
                                kiba_smi_len,
                                log_space=False)
kiba_test_set = DeepDTADataset(kiba_dataset_path, 
                                kiba_test_indices,
                                kiba_protein_len,
                                kiba_smi_len,
                                log_space=False)
kiba_train_loader = DataLoader(dataset=kiba_train_set,
                          batch_size=batch_size,
                          pin_memory=True,
                          shuffle=True) 
kiba_val_loader = DataLoader(dataset=kiba_val_set,
                          batch_size=batch_size,
                          pin_memory=True,
                          shuffle=True)
kiba_test_loader = DataLoader(dataset=kiba_test_set,
                          batch_size=batch_size,
                          pin_memory=True,
                          shuffle=True)

In [ ]:
# Loading Davis dataset                

davis_train_set = DeepDTADataset(davis_dataset_path,
                                davis_train_indices,
                                davis_protein_len,
                                davis_smi_len,
                                log_space=True)
davis_val_set = DeepDTADataset(davis_dataset_path,
                                davis_val_indices,
                                davis_protein_len,
                                davis_smi_len,
                                log_space=True)
davis_test_set = DeepDTADataset(davis_dataset_path,
                                davis_test_indices,
                                davis_protein_len,
                                davis_smi_len,
                                log_space=True)
davis_train_loader = DataLoader(dataset=davis_train_set,
                          batch_size=batch_size,
                          pin_memory=True,
                          shuffle=True) 
davis_val_loader = DataLoader(dataset=davis_val_set,
                          batch_size=batch_size,
                          pin_memory=True,
                          shuffle=True)
davis_test_loader = DataLoader(dataset=davis_test_set,
                          batch_size=batch_size,
                          pin_memory=True,
                          shuffle=True)

In [ ]:
# reading a batch

for data in davis_train_loader:
    xd, xt, y = data 
    print(xd.shape)
    print(xt.shape)
    print(y.shape)
    break

In [ ]:
# reading a batch

for data in kiba_train_loader:
    xd, xt, y = data 
    print(xd.shape)
    print(xt.shape)
    print(y.shape)
    break

### 4.2 Model

In [ ]:
class DrugPredictionLSTM(nn.Module):    
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_size, n_layers, drop_prob=0.3):        
        super().__init__()
        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
    

    def forward(self, x):        
        batch_size = x.size(0)        
        embeds = self.embedding(x)        
        lstm_out, hidden = self.lstm(embeds)        
        hidden = hidden[0][-1].view(-1, self.hidden_dim)
        out = self.dropout(hidden)
        out = self.fc(out)
        sig_out = self.sig(out)
        return sig_out.view(-1)

In [ ]:
input_size = 100
embedding_dim = 128
output_size = 1
hidden_dim = 256
n_layers = 2

net = DrugPredictionLSTM(input_size, embedding_dim, hidden_dim, output_size, n_layers)
net = net.to(device)
print(net)

### 4.3 Plots & Evaluation Functions

In [ ]:
# taken from https://github.com/hkmztrk/DeepDTA/blob/master/source/datahelper.py

# CI Score

def cindex_score(y_true, y_pred):
    g = tf.subtract(tf.expand_dims(y_pred, -1), y_pred)
    g = tf.cast(g == 0.0, tf.float32) * 0.5 + tf.cast(g > 0.0, tf.float32)
    f = tf.subtract(tf.expand_dims(y_true, -1), y_true) > 0.0
    f = tf.matrix_band_part(tf.cast(f, tf.float32), -1, 0)
    g = tf.reduce_sum(tf.multiply(g, f))
    f = tf.reduce_sum(f)
    return tf.where(tf.equal(g, 0), 0.0, g/f) #select

In [ ]:
def plot_loss(train_log, val_log):
    plt.plot(range(1, epoch+1), train_log, color='C0', label='training')
    plt.plot(range(1, epoch+1), val_log, color='C1', label='validation')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch Number')
    plt.ylabel('Loss')
    plt.legend(loc='best')
    plt.show()

In [ ]:
def plot_loss(train_ci_score, val_ci_score):
    plt.plot(range(1, epoch+1), train_ci_score, color='C0', label='training')
    plt.plot(range(1, epoch+1), val_ci_score, color='C1', label='validation')
    plt.title('Training and Validation CI Score')
    plt.xlabel('Epoch Number')
    plt.ylabel('CI Score')
    plt.legend(loc='best')
    plt.show()

### 4.4 Training & Test

In [ ]:
epoch_num = 100
lr=0.001
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

In [ ]:
def train(net, train_loader, val_loader):
    train_log = []
    val_log = []

    for epoch in range(1, epoch_num+1):
        running_loss = 0    
        train_loss = []
        net.train()
        for (inputs, labels) in tqdm(train_loader, desc='Training epoch ' + str(epoch), leave=False):        
            inputs, labels = inputs.to(device), labels.to(device)        
            optimizer.zero_grad()
            outputs = net(inputs.float())
            loss = criterion(outputs, labels)       
            loss.backward()                
            optimizer.step()        
            train_loss.append(loss.item())
        train_log.append(np.mean(train_loss))

        running_loss = 0
        test_loss = []
        net.eval()
        with torch.no_grad():                
            for (inputs, labels) in tqdm(val_loader, desc='Validation ', leave=False):         
                inputs, labels = inputs.to(device), labels.to(device)        
                outputs = net(inputs.float())                       
                loss = criterion(outputs, labels)            
                test_loss.append(loss.item())
        test_log.append(np.mean(test_loss))  
    return train_log, val_log  

In [ ]:
def test(net, test_loader):
    y_true = []
    y_pred = []
    with torch.no_grad():
        lstm_classifier.eval()
        for (inputs, labels) in tqdm(test_loader, desc='Test'):         
            inputs, labels = inputs.to(device), labels.to(device)        
            outputs = lstm_classifier(inputs.float())                       
            y_true += torch.eye(num_classes)[labels].tolist()
            y_pred += outputs.tolist()
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return y_true, y_pred

#### 4.4.1 KIBA dataset

In [ ]:
kiba_train_log, kiba_val_log = train(net, kiba_train_loader, kiba_val_loader)

In [ ]:
plot_loss(kiba_train_log, kiba_val_log)

In [ ]:
y_true_kiba, y_pred_kiba = test(net, kiba_test_loader)
print("KIBA Test CI Score: ", cindex_score(y_true_kiba, y_pred_kiba))
print("KIBA Test MSE: ", mean_squared_error(y_true_kiba, y_pred_kiba))

#### 4.4.2 Davis dataset

In [ ]:
davis_train_log, davis_val_log = train(net, davis_train_loader, davis_val_loader)

In [ ]:
plot_loss(davis_train_log, davis_val_log)

In [ ]:
y_true_davis, y_pred_davis = test(net, davis_test_loader)
print("Davis Test CI Score: ", cindex_score(y_true_davis, y_pred_davis))
print("Davis Test MSE: ", mean_squared_error(y_true_davis, y_pred_davis))

## 5. Classification

In [ ]:
davis_threshold = 7 
kiba_threshold = 12.1

In [ ]:
y_pred_binary_davis = (y_pred_davis > davis_threshold) * 1
y_true_binary_davis = (y_true_davis > davis_threshold) * 1
y_pred_binary_kiba = (y_pred_kiba > kiba_threshold) * 1
y_true_binary_kiba = (y_true_kiba > kiba_threshold) * 1

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true_binary_davis, y_pred_binary_davis).ravel()

davis_accuracy = (tp + tn) / (tp + tn + fp + fn)
davis_sensitivity = tp / (tp + fn)
davis_specificity = tn / (tn + fp)
davis_f1_score = tp / (tp + 1/2 * (fp + fn))

print("=== Davis Statistics ===")
print("Test Accuracy: %.3f" % davis_accuracy)
print("Test Sensitivity: %.3f" % davis_sensitivity)
print("Test Specificity: %.3f" % davis_specificity)
print("Test F1-Score: %.3f" % davis_f1_score)

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true_binary_kiba, y_pred_binary_kiba).ravel()

kiba_accuracy = (tp + tn) / (tp + tn + fp + fn)
kiba_sensitivity = tp / (tp + fn)
kiba_specificity = tn / (tn + fp)
kiba_f1_score = tp / (tp + 1/2 * (fp + fn))

print("=== KIBA Statistics ===")
print("Test Accuracy: %.3f" % kiba_accuracy)
print("Test Sensitivity: %.3f" % kiba_sensitivity)
print("Test Specificity: %.3f" % kiba_specificity)
print("Test F1-Score: %.3f" % kiba_f1_score)


## 6. Binding Histogram

In [ ]:
def plot_binding_histogram(train_loader, val_loader, test_loader, data_name):
    all_bindings = []

    for data in train_loader:
        _, _, y = data
        all_bindings.extend(y) 
    for data in val_loader:
        _, _, y = data
        all_bindings.extend(y)    
    for data in test_loader:
        _, _, y = data
        all_bindings.extend(y)

    all_bindings = torch.stack(all_bindings)

    plt.figure(figsize=(7,7))
    plt.title(data_name + " Bindings Histogram")
    plt.xlabel("Binding Value")
    plt.ylabel("Count")
    plt.hist(all_bindings, bins=20)

In [ ]:
# Davis dataset

plot_binding_histogram(davis_train_loader, davis_val_loader, davis_test_loader, "Davis")

In [ ]:
# KIBA dataset

plot_binding_histogram(kiba_train_loader, kiba_val_loader, kiba_test_loader, "KIBA")

**Bias Issue**: As we can see in the histograms above, the binding values in Davis dataset are most concentrated in the region close to 5 and the binding values in KIBA dataset are most concentrated in area between 11 and 12.  Therefore, in the model trained on Davis dataset, the outputs may become biased towards value 5 and in the model trained on KIBA dataset, the binding values may become biased toward 11. 

**Solution**: In order to prevent this balance issue, we can use data augmentation or weighted resampling techniques to use the samples whose binding values are rare more than other samples. This will create a less-biased histogram and can potentially help the model with its predictions.

Another way to mitigate this problem is to use evaluation metrics that are suitable for highly unbalanced datasets. One of this metrics is AUPR (Area Under the Precision Recall curve) that punishes false positive predictions more than metrics like MSE [3]. 

## 7. Choosing the best model

## 8. Interpreting the model

In this section, we introduce four methods that can help in interpreting neural networks. 

### 8.1. Saliency Maps

### 8.2. Guided Back Propagation

### 8.3. Guided Grad-Cam

### 8.4. LRP

## 9. Robustness & Accuracy

## 10. Meaning of Repetitions

## 11. Making the model more interpretable

## 12. References & Acknowledgements

[1. Comparison Study of Computational Prediction Tools for Drug-Target
Binding Affinities](https://www.frontiersin.org/articles/10.3389/fchem.2019.00782/full)

[2. DeepDTA: deep drug–target binding affinity prediction](https://arxiv.org/abs/1801.10193)

[3. SimBoost: a read-across approach
for predicting drug–target binding afnities
using gradient boosting machines](https://jcheminf.biomedcentral.com/track/pdf/10.1186/s13321-017-0209-z)

[4. Deep Inside Convolutional Networks: Visualising
Image Classification Models and Saliency Maps](https://arxiv.org/pdf/1312.6034.pdf)